#### WorldBank TCdata360

In [5]:
import pandas as pd
import numpy as np
import os
import time
import logging
from datetime import datetime
import requests
from bs4 import BeautifulSoup
import openpyxl
import json
import xmltodict
import io

pd.set_option('display.max_colwidth', None)
#logger
logging.basicConfig(filename='ISPAR.log', filemode='w', format='%(asctime)s - %(message)s', level=logging.INFO)

#### getting Pillar Performance Per Country

In [6]:
id={3:'Global Cybersecurity Index',
        4:'Global Gender Gap Index',
        5:'ICT Development Index',
        6:'Network Readiness Index',
        7:'Global Innovation Index',
        9:'E-Government Development Index',
        11:'Global Competitiveness Index',
        12:'E-Participation index',
        14:'AI Readiness Index',
        16:'Open Data Inventory Index'}

#get country list
country=['Algeria','Bahrain','Comoros','Djibouti','Egypt','Iraq','Jordan','Kuwait','Lebanon','Libya','Morocco','Oman','Qatar','Saudi Arabia','Somalia','State of Palestine','Sudan','Syrian Arab Republic','Tunisia','United Arab Emirates','Yemen, Rep.']

url='https://datacatalog.unescwa.org/datastore/dump_v2?resource_id=e0d88222-f90c-4a92-8de4-397f6529c402&format=csv'

#get the url for download
response=requests.get(url)

try:
    if response.status_code!=200:
        logging.warning(f'!!!!!!WARNING!!!!!!status code is {response.status_code}')

    #read the response into pandas dataframe
    df = pd.read_csv(io.BytesIO(response.content))
    logging.info(f'dataframe returned with {len(df)} rows')

    #remove the 0.colname from column names
    df.columns=[i.replace("0.", "") for i in df.columns]

    #filter on Index ID and 'Country Name'
    df_filtered=df[(df['Index ID'].isin(id)) & (df['Country Name'].isin(country))]

    df_filtered.to_excel('ISPAR.xlsx', index=False)
    logging.info(f'dataframe for Arab countries returned with {len(df_filtered)} rows')

    #goupby and aggregate as mean
    ispar_aggregated=df_filtered.groupby(['Index Name', 'Year']).aggregate({'Score':'mean'})
    ispar_aggregated.to_excel('ISPAR_AGGREGATED.xlsx')
    logging.info('groupby.aggregate() successful and saved as ISPAR_AGGREGATED.xlsx')

    # #to download to a csv
    # with open('ISPAR.csv', 'wb') as f:
    #     f.write(response.content)

except Exception as e:
    logging.warning(f'!!!WARNING!!!  {e}')

In [7]:
df_filtered.head()

,Country Name,ISO,Index ID,Index Name,Pillar ID,Pillar Name,Rank,Score,Year,_id
79,Iraq,IRQ,12,E-Participation index,75,Stage Two,91,0.2143,2022,700061
85,Jordan,JOR,12,E-Participation index,75,Stage Two,60,0.4286,2022,700067
89,Kuwait,KWT,12,E-Participation index,75,Stage Two,60,0.4286,2022,700071
93,Lebanon,LBN,12,E-Participation index,75,Stage Two,113,0.1429,2022,700075
96,Libya,LBY,12,E-Participation index,75,Stage Two,135,0.0714,2022,700078
